In [1]:
#include <type_traits>

namespace std {
inline namespace adobe_dcx {
    
template <class T>
class optional {
    typename std::aligned_storage<sizeof(T)>::type _storage;
    bool _has_value = false;
    
public:
    using value_type = T;
    
    constexpr optional() noexcept = default;
    
    constexpr optional(const optional& other) {
        if (!other.has_value()) {
        } else if (has_value()) {
            **this = *other;
        } else {
            new (&_storage) T(*other);
            _has_value = true;
        }
    }
    
    optional(optional&& other) noexcept(std::is_nothrow_move_constructible<T>::value && std::is_nothrow_move_assignable<T>::value) {
        if (!other.has_value()) {
            reset();
        } else if (has_value()) {
            **this = std::move(*other);
        } else {
            new (&_storage) T(std::move(*other));
            _has_value = true;
        }
    }
    
    template <class U = value_type>
    constexpr optional(U&& value) {
        new (&_storage) T(std::forward<U>(value));
        _has_value = true;
    }
    
    ~optional() {
        if (has_value()) (**this).T::~T();
    }
    
    void reset() noexcept {
        if (!has_value()) return;
        (**this).T::~T();
        _has_value = false;
    }
    
    T& operator*() { return reinterpret_cast<T&>(_storage); }
    const T& operator*() const { return reinterpret_cast<const T&>(_storage); }
    
    constexpr bool has_value() const noexcept {
        return _has_value;
    }
    
    constexpr explicit operator bool() const noexcept {
        return _has_value;
    }
    
    template <class U>
    constexpr T value_or(U&& default_value) const& {
        return has_value() ? (**this) : static_cast<T>(std::forward<U>(default_value));
    }
    template <class U>
    constexpr T value_or(U&& default_value) && {
        return has_value() ? std::move(**this) : static_cast<T>(std::forward<U>(default_value));
    }
};
    
}
}

In [2]:
using namespace std;
optional<int> x = 10;

In [3]:
*x

10

In [4]:
x.has_value()

true

In [5]:
x.value_or(42)

10

In [6]:
x.reset();

In [7]:
x.has_value()

false

In [8]:
x.value_or(42)

42

Thought on tracked values -

A tracked value should be bound to an execution context. You want to be able to write:
```cpp
tracked | [](auto& x){
    x.stuff();
};
```
vs.
```cpp
async(executor, [] {
    if (auto p = tracked.lock()) {
        p->stuff();
    }
});
```

Should work with optional (and futures?) as well. Also, this would be when/if_all - requiring the same executor (how can you test that? - no equality on lambdas!)

```cpp
(track1, track2) | core | [](auto& x1, auto& x2){
       // use x1 an x2
} | surface | ;
```